In [1]:
import pandas as pd
import numpy as np

from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import great_circle
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm_base, tqdm_notebook as tqdm

tqdm_base.pandas()

In [16]:
import os
import gtfs_transformer as gt

In [2]:
%load_ext autoreload
%autoreload 2

## Read positions from August 2018

In [4]:
locations_df = pd.read_hdf('../data/intermed/positions_201808.h5', key='df')

In [5]:
locations_df.head()

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct
timestamp_utc,,,,,,,,
2018-07-31 23:55:06,100495,1533081306,39570076,8109,47.606529,-122.322571,2018-07-31 23:55:06,2018-07-31 16:55:06-07:00
2018-07-31 23:55:06,100128,1533081306,39494444,1154,47.754990,-122.155212,2018-07-31 23:55:06,2018-07-31 16:55:06-07:00
2018-07-31 23:55:07,102574,1533081307,39515435,7006,47.599838,-122.329063,2018-07-31 23:55:07,2018-07-31 16:55:07-07:00
2018-07-31 23:55:09,100061,1533081309,39571262,7167,47.383316,-122.230255,2018-07-31 23:55:09,2018-07-31 16:55:09-07:00
2018-07-31 23:55:31,100146,1533081331,39496283,6900,47.596931,-122.328285,2018-07-31 23:55:31,2018-07-31 16:55:31-07:00


In [6]:
locations_df.size

79683368

In [7]:
locations_df.shape

(9960421, 8)

In [8]:
pd.to_datetime('August 2018')

Timestamp('2018-08-01 00:00:00')

In [11]:
os.listdir('..')

['01_transform_source_data.ipynb',
 '03_e_segment_analysis.html',
 '.DS_Store',
 'unpack_pb_files.py',
 '04_transform_d_locations.ipynb',
 '02_transform_e_locations.html',
 'data_transformation',
 'README.md',
 'download_raw_locations.sh',
 '.gitignore',
 'route_shape_process',
 '03_e_segment_analysis.ipynb',
 '02_transform_e_locations.ipynb',
 '.ipynb_checkpoints',
 '.git',
 'data']

In [15]:
for entry in os.scandir('..'):
    print(entry.name, entry.path)

01_transform_source_data.ipynb ../01_transform_source_data.ipynb
03_e_segment_analysis.html ../03_e_segment_analysis.html
.DS_Store ../.DS_Store
unpack_pb_files.py ../unpack_pb_files.py
04_transform_d_locations.ipynb ../04_transform_d_locations.ipynb
02_transform_e_locations.html ../02_transform_e_locations.html
data_transformation ../data_transformation
README.md ../README.md
download_raw_locations.sh ../download_raw_locations.sh
.gitignore ../.gitignore
route_shape_process ../route_shape_process
03_e_segment_analysis.ipynb ../03_e_segment_analysis.ipynb
02_transform_e_locations.ipynb ../02_transform_e_locations.ipynb
.ipynb_checkpoints ../.ipynb_checkpoints
.git ../.git
data ../data


In [19]:
!ls ../data/source/

gtfs_20180118 gtfs_20180611 gtfs_20180718
gtfs_20180511 gtfs_20180717 gtfs_20180815


In [26]:
gtfs_aug18 = gt.GTFS('../data/source/gtfs_20180815')

In [27]:
gtfs_aug18.__dict__.keys()

dict_keys(['fare_attributes', 'block_trip', 'agency', 'fare_rules', 'calendar_dates', 'stop_times', 'shapes', 'trips', 'stops', 'block', 'calendar', 'routes'])

In [25]:
'abcdefg'[:-3]

'abcd'

In [28]:
gtfs_aug18.routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,100001,KCM,1,NaN,Kinnear - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/001/n0.html,NaN,NaN
1,100002,KCM,10,NaN,Capitol Hill - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/010/n0.html,NaN,NaN
2,100003,KCM,101,NaN,Renton Transit Center - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/101/n0.html,NaN,NaN
3,100004,KCM,105,NaN,Renton Highlands - Renton Transit Center,3,http://metro.kingcounty.gov/schedules/105/n0.html,NaN,NaN
4,100005,KCM,106,NaN,Renton Transit Center - Skyway - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/106/n0.html,NaN,NaN


In [29]:
gtfs_aug18.stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone
0,1000,NaN,Pine St & 9th Ave,NaN,47.613415,-122.332138,21,NaN,0,NaN,America/Los_Angeles
1,10000,NaN,NE 55th St & 43rd Ave NE,NaN,47.668575,-122.283653,1,NaN,0,NaN,America/Los_Angeles
2,10005,NaN,40th Ave NE & NE 51st St,NaN,47.665886,-122.284897,1,NaN,0,NaN,America/Los_Angeles
3,10010,NaN,NE 55th St & 39th Ave NE,NaN,47.668579,-122.285667,1,NaN,0,NaN,America/Los_Angeles
4,10020,NaN,NE 55th St & 37th Ave NE,NaN,47.668579,-122.288300,1,NaN,0,NaN,America/Los_Angeles
